In [ ]:
import numpy as np
import csv
import zipfile
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math
from numpy import linalg as LA
from random import shuffle
import cvxpy as cp
import math
from collections import Counter
from statistics import mean
from sklearn.model_selection import GridSearchCV
import warnings
import random
import copy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso,LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import seaborn as sns
%matplotlib inline
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [ ]:
# data generator

def iid_data(d, D, n, sigma):
    # matrix X generation
    XX = np.zeros((n, d + 1))
    for i in range(n):
        XX[i][0] = 1.0
        XX[i][1:] = np.random.normal(0, 1, d)

    # D/d sparse w generation
    w = [random.uniform(-1.0, 1.0) for i in range(D)]
    w = np.array(w)
    w = w / math.pow(LA.norm(w, 2), 2)
    main_dim = set([])
    while main_dim.__len__() < D:
        main_dim.add(random.randint(1, d - 1))
    main_dim = list(main_dim)
    main_dim.sort()
    ww = np.zeros((d + 1,))
    for i in range(D):
        ww[main_dim[i]] = w[i]

    # error generation
    error = np.random.normal(0, sigma, n)

    # Y generation
    YY = [np.matmul(np.transpose(ww), XX[i]) + error[i] for i in range(n)]
    YY = np.array(YY)

    return XX, YY, ww, main_dim,


def partition(XX, YY, N):
    X = []
    Y = []
    nn = int(XX.__len__() / N)
    for i in range(N):
        X.append(np.array(XX[i * nn:(i + 1) * nn]))
        Y.append(np.array(YY[i * nn:(i + 1) * nn]))

    return X, Y

In [ ]:
# evaluation

def MSE(XX, YY, w):
    return LA.norm(YY - np.matmul(XX, w), 2) / XX.__len__()

def FMeasure(w, main_dim):
    dim = []
    for i in range(w.__len__()):
        if w[i] != 0:
            dim.append(i)
    r = (set(dim) & set(main_dim)).__len__() / main_dim.__len__()
    p = (set(dim) & set(main_dim)).__len__() / (dim.__len__() + 0.000000001)

    return r,p, dim.__len__()/w.__len__()

def similarity(Ws, main_dim):
  dims = []
  MV = [0 for i in range(d+1)]
  for j in range(N):
    dims.append([])
    for i in range(d+1):
      if Ws[j][i] != 0:
        dims[j].append(i)
        MV[i] += 1
  a = [i for i in range(1001)]
  for i in range(N):
    a  = (set(a) & set(dims[i]))
  print('\n Intersection')
  print('intersect size: ' + str(a.__len__()))
  print((set(a) & set(main_dim)).__len__())


def MJ(ws):
  whole = []
  output = []
  for i in range(dims.__len__()):
    for j in range(dims[i].__len__()):
      whole.append(dims[i][j])

  for i in range(20):
      a = Counter(whole).most_common(1)[0][0]
      output.append(a)
      whole = list(filter((a).__ne__, whole))

  print((set(output) & set(main_dim)).__len__())

In [ ]:
# some functions need for the paper

def generating_M(X):
    print('generating covariance matrix is on progress')  
    X = list(X)
    for i in range(X.__len__()):
        X[i] = list(X[i])
    for row in X:
            del row[0]
    gama = []
    # lambda_hat = [math.pow(10,i) * math.sqrt(math.log10(d)/( X.__len__())) for i in range(5)]
    lambda_hat = [math.pow(10, i-10) for i in range(15)]
    print('first for')
    for i in range(d):
        if i%10==0:
          print("*", end = '')
        x = [row[i] for row in X]
        XX = copy.deepcopy(X)
        for row in XX:
            del row[i]
        XX = np.array(XX)
        out = centralized(XX,x, lambda_hat, 10000)
        gama.append(out[0])

    C_hat = np.ones((d, d))
    print('\n senond for')
    for i in range(d):
        if i%10==0:
          print("*", end = '')      
        for j in range(d):
            if j < i:
              C_hat[i][j] = -1 * gama[i][j]
            elif i == j:
                C_hat[i][j] = 1
            else:
              C_hat[i][j] = -1 * gama[i][j-1]
    tai = []
    print('\n third for')
    for i in range(d):
        if i%10==0:
          print("*", end = '')
        x = [row[i] for row in X]
        XX = copy.deepcopy(X)
        for j in XX:
            del j[i]
        a = np.transpose(x - np.matmul(XX, gama[i]))
        a = np.matmul(a,x)
        tai.append(a / X.__len__())
    T_hat = np.diag(tai)
    temp = np.matmul(T_hat, C_hat)
    M = np.zeros((d+1,d+1))
    for i in range(d+1):
      for j in range(d+1):
        if i==0 or j == 0:
          if i == 0 and j == 0:
            M[i][j] = 1.0
        else:
          M[i][j] = temp[i-1][j-1]

    print('generating covariance matrix finished')  
    return M

In [ ]:
# optimizer

def centralized(X, Y, lambdas, itr):
    lasso = Lasso(alpha=0.1, max_iter=10000)
    parameters = {
        'alpha': lambdas}
    lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=10)
    lasso_regressor.fit(X, Y)
    lasso = Lasso(alpha=lasso_regressor.best_params_['alpha'], max_iter=itr)
    lasso.fit(X, Y)
    w = list(lasso.coef_)
    return w, lasso.alpha

def debiased(w, Y, X):
    w_d = Y - np.matmul(X, w)
    w_d = np.matmul(np.transpose(X), w_d)
    pre = generating_M(X)
    w_d = (np.matmul(pre, w_d)) / X.__len__()
    w_d = w + w_d
    return w_d

In [ ]:
# generating data
d = 999
D = 20
n = 2000
sigma = 0.00001
X, Y, main_w, main_dim = iid_data(d, D, n, sigma)

# save data in to a csv-file
np.savetxt('X.csv', X, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
np.savetxt('Y.csv', Y, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
np.savetxt('main_w.csv', main_w, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
np.savetxt('main_dim.csv', main_dim, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)

In [ ]:
# retrieve data

X = genfromtxt('X.csv', delimiter=' ')
Y = genfromtxt('Y.csv', delimiter=' ')
main_w = genfromtxt('main_w.csv', delimiter=' ')
main_dim = genfromtxt('main_dim.csv', delimiter=' ')

In [ ]:
# centralized (Lasso)

lambdas = [math.pow(10, i-10) for i in range(20)]
w_c_l, alpha = centralized(X, Y, lambdas, 100000)

# evaluation (MSE, R2, Recall=(Power), Precision(=1-FDR), sparsity)
MSE_c_l = MSE(X, Y, w_c_l)

f = [np.matmul(np.transpose(w_c_l), X[i]) for i in range(X.__len__())]
R2_c_l = r2_score(Y, f)

R_c_l, P_c_l, S_c_l = FMeasure(w_c_l, main_dim)

# save results
f = open("Centralized_Lasso_Results.txt", "a")
f.write('MSE: ' + str(MSE_c_l))
f.write('\nR2: ' + str(R2_c_l))
f.write('\nPrecision: ' + str(P_c_l))
f.write('\nRecall: ' + str(R_c_l))
f.write('\nSparsity: ' + str(S_c_l))
f.close()

In [ ]:
# centralized (de-biased Lasso)

lambdas = [math.pow(10, i-10) for i in range(20)]
w_c_d = debiased(w_c_l, Y, X)

# evaluation (MSE, R2, Recall=(Power), Precision(=1-FDR), sparsity)
MSE_c_d = MSE(X, Y, w_c_d)

f = [np.matmul(np.transpose(w_c_d), X[i]) for i in range(X.__len__())]
R2_c_d = r2_score(Y, f)

P_c_d, R_c_d, S_c_d = FMeasure(w_c_d, main_dim)

# save results
f = open("Centralized_DBLasso_Results.txt", "a")
f.write('MSE: ' + str(MSE_c_d))
f.write('\nR2: ' + str(R2_c_d))
f.write('\nPrecision: ' + str(P_c_d))
f.write('\nRecall: ' + str(R_c_d))
f.write('\nSparsity: ' + str(S_c_d))
f.close()

In [ ]:
# partitioning
N = 20
X_train, Y_train = partition(X, Y, N)

In [ ]:
# de-centralized (Lasso)

lambdas = [math.pow(10, i - 10) for i in range(20)]
wls = []
for i in range(N):
    print('client ' + str(i) + ' is in progress.')
    out = centralized(X_train[i], Y_train[i], lambdas, 100000)
    wls.append(out[0])

# evaluation (MSE, R2, Recall=(Power), Precision(=1-FDR), sparsity)
MSE_d_l = 0
R2_d_l = 0
R_d_l = 0
P_d_l = 0
S_d_l = 0
for i in range(N):
    MSE_d_l += MSE(X_train[i], Y_train[i], wls[i])
    f = [np.matmul(np.transpose(wls[i]), X_train[i][j]) for j in range(X_train[i].__len__())]
    R2_d_l += r2_score(Y_train[i], f)
    out = FMeasure(wls[i], main_dim)
    R_d_l += out[0]
    P_d_l += out[1]
    S_d_l += out[2]
MSE_d_l = MSE_d_l / N
R2_d_l = R2_d_l / N
R_d_l = R_d_l / N
P_d_l = P_d_l / N
S_d_l = S_d_l / N

# save results
f = open("De-centralized_Lasso_Results.txt", "a")
f.write('Averaged MSE: ' + str(MSE_d_l))
f.write('\nAveraged R2: ' + str(R2_d_l))
f.write('\nAveraged Precision: ' + str(P_d_l))
f.write('\nAveraged Recall: ' + str(R_d_l))
f.write('\nAveraged Sparsity: ' + str(S_d_l))
f.close()

wls = np.array(wls)
np.savetxt('wls.csv', wls, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)


In [ ]:
# de-centralized (de-biased Lasso)

lambdas = [math.pow(10, i-10) for i in range(20)]
wds = []
for i in range(N):
  print('client ' + str(i) + ' is in progress.')
  out = debiased(wls[i], Y_train[i], X_train[i])
  wds.append(out[0])


# evaluation (MSE, R2, Recall=(Power), Precision(=1-FDR), sparsity)
MSE_d_d = 0
R2_d_d = 0
R_d_d = 0
P_d_d = 0
S_d_d = 0
for i in range(N):
  MSE_d_d += MSE(X_train[i], Y_train[i], wds[i])
  f =  np.matmul(np.transpose(wds[i]), X_train[i][j]) for j in range(X_train[i].__len__())]
  R2_d_d += r2_score(Y_train[i], f)
  out = FMeasure(wds[i], main_dim)
  R_d_d += out[0]
  P_d_d += out[1]
  S_d_d += out[2]
MSE_d_d = MSE_d_d / N
R2_d_d = R2_d_d / N
R_d_d = R_d_d / N
P_d_d = P_d_d / N
S_d_d = S_d_d / N

# save results
f = open("De-centralized_DBLasso_Results.txt", "a")
f.write('Averaged MSE: ' + str(MSE_d_d))
f.write('\nAveraged R2: ' + str(R2_d_d))
f.write('\nAveraged Precision: ' + str(P_d_d))
f.write('\nAveraged Recall: ' + str(R_d_d))
f.write('\nAveraged Sparsity: ' + str(S_d_d))
f.close()

wds = np.array(wds)
np.savetxt('wds.csv', wds, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
